## Redo pangenome stuff with new sourmash pangenome plugin. 
- Clean up this repo
- try pangenome in protein space using translated sketches (use 10 v 10 now)
- need to deduplicate the genomes at 99% ANI, then run Roary
- Use roary, then sourmash

In [ ]:
# update plugin
mamba activate pangenomics_dev

# upgrade
pip install --upgrade --force-reinstall git+https://github.com/dib-lab/sourmash_plugin_pangenomics.git


## Steps + What do we want. 
I want to be able to compare roary output with the metagenomes. I want to do this in protein space, as roary works in protein space and that seems like the right comparison. 
I also want to do it in DNA space. I'd like to find a species, that has new strains in pig, but also a reasonable amount of reference strains. 

### Steps:
- Find a species
- Download all strains from genbank/gtdb
- dereplicate at 99% ANI
- run prokka
- Run roary
- Translate 10 metaGs
- make a cloud and a core sketch. 
- Run these against the metagenomes

In [ ]:
# where is the tax file:
/group/ctbrowngrp2/scratch/annie/2023-swine-sra/results/MAGs/250411_mag_taxonomy.tsv

In [ ]:
# Do a test with just reference genomes. (Megasphaera elsdenii)
mkdir test_pipeline

# see Snakefile
# how to go from 2 lists of species to a folder with all of them: For drep. 
srun --account=ctbrowngrp -p med2 -J roary -t 1:00:00 -c 10 --mem=50gb --pty bash
mamba activate branchwater-skipmer

snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 10 --rerun-incomplete -k -n

In [ ]:
# now all renamed to .fasta: how to run dRep?
mamba activate dRep
# Run dRep at 95% ANI over 85% of length of longest contigs
dRep dereplicate ./drep --S_algorithm ANImf \
--ignoreGenomeQuality -l 10000 -sa 0.95 -nc 0.85 \
-g ./250418_viral_sequences.genomad.fa.split/*.fa -p 48